# Module 2: XBase Routing API

## What you will learn
* XBase layout engine routing grid systems
* Simple APU for creating tracks, vias, pins, etc.
* TemplateBase self.size and self.bound_box attributes

## XBase Routing Grid

```yaml
routing_grid:
    layers: [4, 5, 6, 7]
    spaces: [0.084, 0.100, 0.084, 0.100]
    bot_dir: 'x'
```
* Defined in specification file.
    * In demp_specs/demo.yaml
* Track direction must alternate between layers
*  Different layers can have different pitch
* A layout must contain integer number of tracks on all layers

## XBase Routing Tracks

<img src="bootcamp_pics/2_xbase_routing/xbase_routing_1.PNG" />
* Track pitch is sum of width and space
* Track 0 is half-pitch from boundary
* Track locations are quantized
<img src="bootcamp_pics/2_xbase_routing/xbase_routing_2.PNG" />
* Draw wider wires by using multiple tracks
    * Wire width is quantized
* Issues:
    * Even width wire wastes space (width=2 takes up same space as width=3)
    * Cannot share tracks with adjacent blocks
<img src="bootcamp_pics/2_xbase_routing/xbase_routing_3.PNG" />
* Solution: allow half-integer tracks
* Track -0.5 is right on the left/bottom boundary

## TrackID and WireArray
```python
class TrackID(object):
    def __init__(self, layer_id, track_idx, width=1, num=1, pitch=0.0):
        #type: (int, Union[float, int], int, int, Union[float, int]) -> None
        
class WireArray(object):
    def __init__(self, track_id, lower, upper):
        #type: (TrackID, float, float) -> None
```
* Routing track location(s) represented by TrackID object
* Physical wire(s) represented by WireArray object
* Layout pins can only be added on WireArray objects
* TrackID and WireArray has built-in support for drawing an array of wires

In [1]:
import sys
sys.path.append('BAG_XBase_demo/demo_scripts')
sys.path.append('BAG_XBase_demo/xbase_demo/demo_layout')
import bootcamp_demo as d
#import core
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
# noinspection PyUnresolvedReferences,PyCompatibility
from builtins import *

from bag.layout.routing import TrackID
from bag.layout.template import TemplateBase

from abs_templates_ec.analog_core import AnalogBase

## BAG Layout Generation Code
```python
def gen_layout(prj, specs, dsn_name):
    # get information from specs
    dsn_specs = specs[dsn_name]
    impl_lib = dsn_specs['impl_lib']
    layout_params = dsn_specs['layout_params']
    lay_package = dsn_specs['layout_package']
    lay_class = dsn_specs['layout_class']
    gen_cell = dsn_specs['gen_cell']

    # get layout generator class
    lay_module = importlib.import_module(lay_package)
    temp_cls = getattr(lay_module, lay_class)

    # create layout template database
    tdb = make_tdb(prj, specs, impl_lib)
    # compute layout
    print('computing layout')
    template = tdb.new_template(params=layout_params, temp_cls=temp_cls)
    # create layout in OA database
    print('creating layout')
    tdb.batch_layout(prj, [template], [gen_cell])
    # return corresponding schematic parameters
    print('layout done')
    return template.sch_params
```

## BAG TemplateDB Creation Code
```python
def make_tdb(prj, specs, impl_lib):
    grid_specs = specs['routing_grid']
    layers = grid_specs['layers']
    spaces = grid_specs['spaces']
    widths = grid_specs['widths']
    bot_dir = grid_specs['bot_dir']

    # create RoutingGrid object
    routing_grid = RoutingGrid(prj.tech_info, layers, spaces, widths, bot_dir)
    # create layout template database
    tdb = TemplateDB('template_libs.def', routing_grid, impl_lib, use_cybagoa=True)
    return tdb
```
* TemplateDB is a database of all layouts created in BAG
* It handles layout hierarchy and is also reponsible for creating layout in OA

## Routing Code Example
* BAG_XBase_demo/demo_layout/core.py
* Look at the below code for the RoutingDemo class, draw_layout() function
* Connecting between layers
    * self.connect_to_tracks() connects one or more wires to tracks on an adjacent layer
    * optimal vias are created automatically
* Adding pins
    * self.add_pin() creates layout pins on WireArrays
    * These pins can be accessed in BAG when doing hierarchical layout
* Size of a layout specified by the self.size attribute
    * Defined as the number of vertical/horizontal tracks on the top two layers

In [2]:
class RoutingDemo_new(TemplateBase):
    """A template of a single transistor with dummies.
    This class is mainly used for transistor characterization or
    design exploration with config views.
    Parameters
    ----------
    temp_db : :class:`bag.layout.template.TemplateDB`
            the template database.
    lib_name : str
        the layout library name.
    params : dict[str, any]
        the parameter values.
    used_names : set[str]
        a set of already used cell names.
    kwargs : dict[str, any]
        dictionary of optional parameters.  See documentation of
        :class:`bag.layout.template.TemplateBase` for details.
    """

    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        super(RoutingDemo_new, self).__init__(temp_db, lib_name, params, used_names, **kwargs)

    @classmethod
    def get_params_info(cls):
        """Returns a dictionary containing parameter descriptions.
        Override this method to return a dictionary from parameter names to descriptions.
        Returns
        -------
        param_info : dict[str, str]
            dictionary from parameter name to description.
        """
        return {}

    def draw_layout(self):
        """Draw the layout of a transistor for characterization.
        """

        # Metal 4 is horizontal, Metal 5 is vertical
        hm_layer = 4
        vm_layer = 5

        # add a horizontal wire on track 0, from X=0.1 to X=0.3
        warr1 = self.add_wires(hm_layer, 0, 0.1, 0.3)
        # print WireArray object
        print(warr1)
        # print lower, middle, and upper coordinate of wire.
        print(warr1.lower, warr1.middle, warr1.upper)
        # print TrackID object associated with WireArray
        print(warr1.track_id)
        # add a horizontal wire on track 1, from X=0.1 to X=0.3,
        # coordinates specified in resolution units
        warr2 = self.add_wires(hm_layer, 1, 100, 300, unit_mode=True)
        # add a horizontal wire on track 2.5, from X=0.2 to X=0.4
        self.add_wires(hm_layer, 2.5, 200, 400, unit_mode=True)
        # add a horizontal wire on track 4, from X=0.2 to X=0.4, with 2 tracks wide
        warr3 = self.add_wires(hm_layer, 4, 200, 400, width=2, unit_mode=True)

        # add 3 parallel vertical wires starting on track 6 and use every other track
        warr4 = self.add_wires(vm_layer, 6, 100, 400, num=3, pitch=2, unit_mode=True)
        print(warr4)

        # create a TrackID object representing a vertical track
        tid = TrackID(vm_layer, 3, width=2, num=1, pitch=0)
        # connect horizontal wires to the vertical track
        warr5 = self.connect_to_tracks([warr1, warr3], tid)
        print(warr5)

        # add a pin on a WireArray
        self.add_pin('pin1', warr1)
        # add a pin, but make label different than net name.  Useful for LVS connect
        self.add_pin('pin2', warr2, label='pin2:')
        # add_pin also works for WireArray representing multiple wires
        self.add_pin('pin3', warr4)
        # add a pin (so it is visible in BAG), but do not create the actual layout
        # in OA.  This is useful for hiding pins on lower levels of hierarchy.
        self.add_pin('pin4', warr3, show=False)

        # set the size of this template
        top_layer = vm_layer
        num_h_tracks = 6
        num_v_tracks = 11
        # size is 3-element tuple of top layer ID, number of top
        # vertical tracks, and number of top horizontal tracks
        self.size = top_layer, num_v_tracks, num_h_tracks
        # print bounding box of this template
        print(self.bound_box)
        # add a M7 rectangle to visualize bounding box in layout
        self.add_rect('M7', self.bound_box)

* Play around with some of the parameters in the above demo class and run the following cell to generate layout in DEMO_ROUTING library

In [3]:
spec_fname = 'demo_specs/demo.yaml'
top_specs = d.read_yaml(spec_fname)

# create BagProject object
local_dict = locals()
if 'bprj' in local_dict:
    print('using existing BagProject')
    bprj = local_dict['bprj']
else:
    print('creating BagProject')
    bprj = d.BagProject()

d.routing_demo(bprj, top_specs, RoutingDemo_new)

creating BagProject
computing layout
WireArray(TrackID(layer=4, track=0), 0.1, 0.3)
0.1 0.2 0.3
TrackID(layer=4, track=0)
WireArray(TrackID(layer=5, track=6, num=3, pitch=2), 0.1, 0.4)
WireArray(TrackID(layer=5, track=3, width=2), 0.042, 0.75)
BBox(0.000, 0.000, 1.980, 0.864)
creating layout
layout done
